In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
# rcParams['figure.figsize'] = 12, 4
import numpy as np
import pandas as pd
from scipy.stats import mode
import xgboost as xgb
from xgboost import XGBRegressor,XGBClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold,StratifiedKFold
from matplotlib import  pyplot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
train = pd.read_csv("../dealinput/train2ok.csv")
test = pd.read_csv("../dealinput/test2ok.csv")
train.head(),test.head()

d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


(                   id  target  qq_bound_已绑定  qq_bound_未绑定  sex_保密  sex_女  \
 0  501951980776722440     0.0             0             1       0      1   
 1  525890212484616200     0.0             0             1       0      0   
 2  599309364691472392     0.0             1             0       1      0   
 3  582559913071350024     0.0             1             0       1      0   
 4  599017959926534408     0.0             0             1       1      0   
 
    sex_男  wechat_bound_已绑定  wechat_bound_未绑定  auth_time_day_1    ...      \
 0      0                 0                 1                0    ...       
 1      1                 1                 0                0    ...       
 2      0                 1                 0                0    ...       
 3      0                 1                 0                0    ...       
 4      0                 1                 0                0    ...       
 
    phone_more_cate_top3_185  phone_more_cate_top3_186  \
 0            

In [139]:
test.id.dtype

dtype('int64')

In [2]:
trainx = train.iloc[:,2:]
trainy = train.iloc[:,1]
print("trainx.shape",trainx.shape)
print("trainy.shape",trainy.shape)
from sklearn.cross_validation import train_test_split
x_train,x_valid,y_train,y_valid = train_test_split(trainx,trainy,test_size = 0.2,random_state = 1)  
print("x_train.shape",x_train.shape)
print("x_valid.shape",x_valid.shape)
print("y_train.shape",y_train.shape)
print("y_valid.shape",y_valid.shape)

trainx.shape (120929, 113)
trainy.shape (120929,)
x_train.shape (96743, 113)
x_valid.shape (24186, 113)
y_train.shape (96743,)
y_valid.shape (24186,)


In [4]:
xgb1 = XGBClassifier(
#         booster = 'dart',
        learning_rate =0.08,
        n_estimators=5000,
        max_depth=5,
        min_child_weight=5,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=3,
        scale_pos_weight=1,
        StratifiedKFold = True,          #0.866124	0.008005	0.965337	0.002286
        seed=1)
xgb_param = xgb1.get_xgb_params()
#构建稀疏矩阵，运行更快
xgtrain = xgb.DMatrix(x_train, label = y_train)
mm = xgb.cv(xgb_param, xgtrain, num_boost_round=1000, nfold=3,
    metrics='auc', early_stopping_rounds=50)
mm.iloc[-3:,:]

# 164
# 0.831457
# 0.008025
# 0.891282
# 0.003152      eta 0.08
#260
# 0.831825
# 0.008380
# 0.892938
# 0.002775    0.05

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
162,0.831414,0.007949,0.890582,0.003258
163,0.831405,0.008054,0.891015,0.003129
164,0.831457,0.008025,0.891282,0.003152


In [69]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True)

def estimate(estimator, name='estimator'):
    auc = cross_val_score(estimator, x_train, y_train, scoring='roc_auc', cv=cv).mean()
    accuracy = cross_val_score(estimator, x_train, y_train, scoring='accuracy', cv=cv).mean()
    recall = cross_val_score(estimator, x_train, y_train, scoring='recall', cv=cv).mean()

    print("{}: auc:{:f}, recall:{:f}, accuracy:{:f}".format(name, auc, recall, accuracy))
    
estimate(XGBClassifier(
                        # booster = 'dart',
                        learning_rate =0.05,
                        n_estimators=mm.shape[0],
                        max_depth=5,
                        min_child_weight=5,
                        gamma=0.1,
                        subsample=0.8,
                        colsample_bytree=0.8,
                        objective= 'binary:logistic',
                        nthread=4,
                        scale_pos_weight=1,
                        StratifiedKFold = True,          #0.866124	0.008005	0.965337	0.002286
                        seed=1),
                         'XGBClassifier') 

print(".......................................")   

d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambig

XGBClassifier: auc:0.833591, recall:0.000365, accuracy:0.971647
.......................................


d:\gongju\python3.6 64\anzhuang\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [6]:
test

,id,qq_bound_已绑定,qq_bound_未绑定,sex_保密,sex_女,sex_男,wechat_bound_已绑定,wechat_bound_未绑定,auth_time_day_1,auth_time_day_4,...,phone_more_cate_top3_185,phone_more_cate_top3_186,phone_more_cate_top3_187,phone_more_cate_top3_188,auth_bank_phoneTop3_same_0,auth_bank_phoneTop3_same_1,bank_name_more_count,card_type_more_count,phone_more_count,phone_num
0,637832512621515016,1,0,1,0,0,1,0,0,0,...,0,1,0,0,1,0,3.277598,0.677726,0.766372,1.078032
1,624385123294384392,1,0,1,0,0,1,0,0,0,...,0,0,0,0,1,0,0.746452,-0.582044,-0.147835,-0.190644
2,636912835502280712,0,1,0,0,1,1,0,0,0,...,0,0,0,0,1,0,-0.519122,0.047841,-0.147835,-0.190644
3,638094054000693512,1,0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,-0.519122,-0.582044,-0.604938,-0.613536
4,636732958538993928,1,0,1,0,0,1,0,0,0,...,0,0,0,0,1,0,-0.519122,-0.582044,-0.604938,-0.613536
5,636668423459311624,1,0,1,0,0,1,0,0,0,...,0,1,0,0,1,0,0.746452,0.047841,0.309269,0.232248
6,628171292146667528,1,0,1,0,0,1,0,0,0,...,0,0,1,0,1,0,-0.519122,0.047841,-0.147835,-0.190644
7,623732422298570760,0,1,1,0,0,0,1,0,0,...,0,0,0,0,1,0,-0.519122,-0.582044,-0.604938,-0.613536
8,637388003227996424,0,1,1,0,0,1,0,0,0,...,0,1,0,0,1,0,-0.519122,-0.582044,-0.604938,-0.613536
9,626505930430746888,1,0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,-0.519122,-0.582044,-0.604938,-0.613536


In [7]:
xgb1 = XGBClassifier(
#         booster = 'dart',
        learning_rate =0.08,
        n_estimators=mm.shape[0],
        max_depth=5,
        min_child_weight=5,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        StratifiedKFold = True,          #0.866124	0.008005	0.965337	0.002286
        seed=1)
xgb1.fit(x_train,y_train)
prexgb = xgb1.predict_proba(test.iloc[:,1:])[:,1]
prexgb = np.round(prexgb,4)
# auc = roc_auc_score(y_valid,prexgb)
# print("xgb1 auc",auc)

In [121]:
import random
rand = []
for i in range(36):
    rand.append(random.uniform(0.75, 0.85))
rand,len(rand)

([0.8044562352690792,
  0.768753642722797,
  0.8408676262577913,
  0.8415237344422946,
  0.8068100994954631,
  0.8182261317679971,
  0.7689461716829638,
  0.8478117303727977,
  0.7606916051858902,
  0.8271745977144555,
  0.820861886216408,
  0.7756671140428979,
  0.7711473304597727,
  0.7778959893684809,
  0.7708511521587805,
  0.804961668716729,
  0.7918912350933461,
  0.7877260035821764,
  0.8111784382906994,
  0.8387381602499127,
  0.8039122273613531,
  0.7562019075230461,
  0.8031417412801879,
  0.7970211425400402,
  0.834385541685755,
  0.8337064059293533,
  0.7836206987064078,
  0.829734059531096,
  0.8322794099119767,
  0.8485207382841425,
  0.7877960684851175,
  0.8147013118199946,
  0.812664150294526,
  0.7557108679500762,
  0.8024320403661312,
  0.7932253550237705],
 36)

In [126]:
prexgb = 0
for i in range(36):
    xgb1 = XGBClassifier(
        #         booster = 'dart',
                learning_rate =0.05,
                n_estimators=mm.shape[0],
                max_depth=5,
                min_child_weight=5,
                gamma=0.1,
                subsample=rand[i],
                colsample_bytree=0.8,
                objective= 'binary:logistic',
                nthread=4,
                scale_pos_weight=1,
                StratifiedKFold = True,          #0.866124	0.008005	0.965337	0.002286
                seed=1)
    xgb1.fit(x_train,y_train)
    prexgb += xgb1.predict_proba(test.iloc[:,1:])[:,1]


In [127]:
prexgb = np.round(prexgb/36,4)
prexgb

array([0.0037, 0.0029, 0.1086, ..., 0.0039, 0.0529, 0.0079], dtype=float32)

In [8]:
prexgb = pd.Series(prexgb)
# auc = roc_auc_score(y_valid,prexgb)
# print("xgb1 auc",auc)
ff = pd.concat([test.id,prexgb],axis = 1)
ff.columns = ["id","prob"]
ff.id = ff.id.astype("int64")
ff
# mm = pd.read_csv("../input/test_auth_info.csv")

# mm = mm.merge(ff,how = "inner",on = "id")
# print(mm.isnull().sum())
# print(mm.info(),ff.info())
# mm
# del mm["id_card"]
# del mm["auth_time"]
# del mm["phone"]
# print(".........")
# print(mm.info())
# mm

,id,prob
0,637832512621515016,0.0048
1,624385123294384392,0.0026
2,636912835502280712,0.0944
3,638094054000693512,0.0119
4,636732958538993928,0.0052
5,636668423459311624,0.0013
6,628171292146667528,0.0530
7,623732422298570760,0.0030
8,637388003227996424,0.0738
9,626505930430746888,0.0080


In [175]:
# mm.columns = ["ID","PROB"]

In [9]:
ff.to_csv("../output/result422.csv",index = False)